In [ ]:
!pip install transformers

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import transformers
import re


In [ ]:
print("tensorflow version : ", tf.__version__)

tensorflow version :  2.4.1


In [ ]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  ['10.79.108.114:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.79.108.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.79.108.114:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 16
#epochs = 6

In [ ]:
from google.colab import drive
drive.mount("/GD")

Drive already mounted at /GD; to attempt to forcibly remount, call drive.mount("/GD", force_remount=True).


In [ ]:
df = pd.read_csv('/GD/My Drive/Colab Notebooks/translated_nlp_df_train.csv', encoding = "ISO-8859-1")
test = pd.read_csv('/GD/My Drive/Colab Notebooks/watson_test.csv',encoding = "ISO-8859-1")
#df_translated = pd.read_csv('/GD/My Drive/Colab Notebooks/translated_nlp_df_train.csv',encoding = "ISO-8859-1")
#train, valid = train_test_split(df, test_size=0.25, random_state = 12)
## 0 for entailment, 1 for neutral, 2 for contradiction ###

In [ ]:
df

,premise_en,hypothesis_en,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Little things like these make a huge differenc...,I was trying to accomplish something.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,In role playing as well Opportunities to expre...,Children can see how different ethnic groups are.,1
...,...,...,...
12115,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,2
12116,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,0
12117,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",2
12118,At the west end is a detailed model of the who...,The model temple complex is at the east end.,2


In [ ]:
def remove_dob_qt(text):
  text = text.replace('"', '')
  return text

df['premise_en'] = df['premise_en'].apply(lambda x: remove_dob_qt(x))
df['hypothesis_en'] = df['hypothesis_en'].apply(lambda x: remove_dob_qt(x))

In [ ]:
def remove_name(text):
  text = text.replace('#NAME', '')
  text = text.replace('??', 'u')
  text = text.replace('-', ' ')
  text = text.replace('--', ' ')
  text = text.strip()
  text = re.sub(r'\d', ' number ', text)
  return text

df['premise_en'] = df['premise_en'].apply(lambda x: remove_name(x))
df['hypothesis_en'] = df['hypothesis_en'].apply(lambda x: remove_name(x))

In [ ]:
df

,premise_en,hypothesis_en,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Little things like these make a huge differenc...,I was trying to accomplish something.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,In role playing as well Opportunities to expre...,Children can see how different ethnic groups are.,1
...,...,...,...
12115,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,2
12116,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,0
12117,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",2
12118,At the west end is a detailed model of the who...,The model temple complex is at the east end.,2


In [ ]:
#df = df.reset_index()
df[df['hypothesis_en'] == 'The costumes were only worked by human hands.']['premise_en']

12111    design, draw and sew all of these gorgeous cos...
Name: premise_en, dtype: object

In [ ]:
df['premise_en'].replace('', np.nan, inplace=True)
df['hypothesis_en'].replace('', np.nan, inplace=True)
df.dropna(subset=['premise_en'], inplace=True)
df.dropna(subset=['hypothesis_en'], inplace=True)

In [ ]:
df.shape

(12120, 3)

In [ ]:
tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-large", do_lower_case=True)

In [ ]:
train_encoded = tokenizer(text=list(df.premise_en.values),
                    text_pair=list(df.hypothesis_en.values),
                    add_special_tokens=True,
                    max_length=max_length,
                    truncation=True,
                    padding=True,
                    return_attention_mask=True,
                    return_token_type_ids=True,
                    return_tensors='tf'
                    )

In [ ]:

val_size = int(len(df)*0.33)

dataset = tf.data.Dataset.from_tensor_slices((train_encoded.data, df.label.values))
val_dataset = (dataset.take(val_size).batch(batch_size))
train_dataset = (dataset.skip(val_size).batch(batch_size))

In [ ]:
MODEL_NAME = 'roberta-large'
with tpu_strategy.scope():
    transformer = transformers.TFAutoModel.from_pretrained(MODEL_NAME)

Some layers from the model checkpoint at roberta-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
def create_model():

    input_1 = tf.keras.Input(shape=(max_length,),name='input_ids', dtype='int32')
    input_2 = tf.keras.Input(shape=(max_length,),name='attention_mask', dtype='int32')
    input_3 = tf.keras.Input(shape=(max_length,),name='token_type_ids', dtype='int32')

    #x = transformer((input_1, input_2, input_3))[0]
    #output_1 = tf.keras.layers.Dense(300, activation='relu')(x[:,0,:])
    #output_2 = tf.keras.layers.Dense(100, activation='relu')(output_1)
    #output = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(output_2)

    sequence_output = transformer((input_1, input_2, input_3))[0]
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
    #max_pool = tf.keras.layers.GlobalMaxPooling1D()(sequence_output)
    #concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(avg_pool)
    output_1 = tf.keras.layers.Dense(300, activation='relu')(dropout)
    output_2 = tf.keras.layers.Dense(100, activation='relu')(output_1)
    output = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(output_2)
    
     
    
    model = tf.keras.Model(inputs=(input_1, input_2, input_3), outputs=output)
    return model

In [ ]:
with tpu_strategy.scope():
    model = create_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate =1e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode TFBaseModelOutputWit 355359744   input_ids[0][0]                  
                                                                 attention_mask[0][0]         

In [ ]:
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience= 1,
    min_lr=1e-7
)

history = model.fit(
    train_dataset,
    epochs = 10,
    verbose = 2,
    batch_size = batch_size,
    callbacks=[lr_reduction],
    validation_data=val_dataset
)

Epoch 1/10


508/508 - 236s - loss: 0.7450 - accuracy: 0.6621 - val_loss: 0.5143 - val_accuracy: 0.8045
Epoch 2/10
508/508 - 79s - loss: 0.4282 - accuracy: 0.8444 - val_loss: 0.4970 - val_accuracy: 0.8140
Epoch 3/10
508/508 - 78s - loss: 0.2670 - accuracy: 0.9085 - val_loss: 0.5743 - val_accuracy: 0.8167
Epoch 4/10
508/508 - 79s - loss: 0.1373 - accuracy: 0.9559 - val_loss: 0.6867 - val_accuracy: 0.8250
Epoch 5/10
508/508 - 79s - loss: 0.0830 - accuracy: 0.9749 - val_loss: 0.7784 - val_accuracy: 0.8240
Epoch 6/10
508/508 - 78s - loss: 0.0716 - accuracy: 0.9776 - val_loss: 0.7824 - val_accuracy: 0.8247
Epoch 7/10
508/508 - 79s - loss: 0.0664 - accuracy: 0.9787 - val_loss: 0.7812 - val_accuracy: 0.8265
Epoch 8/10
508/508 - 79s - loss: 0.0639 - accuracy: 0.9805 - val_loss: 0.7883 - val_accuracy: 0.8247
Epoch 9/10
508/508 - 79s - loss: 0.0649 - accuracy: 0.9785 - val_loss: 0.7948 - val_accuracy: 0.8245
Epoch 10/10
508/508 - 79s - loss: 0.0583 - accuracy: 0.9830 - val_loss: 0.8093 - val_accuracy: 0.8252

In [ ]:
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience= 1,
    min_lr=1e-7
)

history = model.fit(
    train_dataset,
    epochs = 10,
    verbose = 2,
    batch_size = batch_size,
    callbacks=[lr_reduction],
    validation_data=val_dataset
)

Epoch 1/10
508/508 - 78s - loss: 0.0617 - accuracy: 0.9808 - val_loss: 0.8074 - val_accuracy: 0.8257
Epoch 2/10
508/508 - 79s - loss: 0.0579 - accuracy: 0.9817 - val_loss: 0.8164 - val_accuracy: 0.8247
Epoch 3/10
508/508 - 79s - loss: 0.0551 - accuracy: 0.9824 - val_loss: 0.8148 - val_accuracy: 0.8270
Epoch 4/10
508/508 - 79s - loss: 0.0569 - accuracy: 0.9829 - val_loss: 0.8279 - val_accuracy: 0.8257
Epoch 5/10
508/508 - 79s - loss: 0.0529 - accuracy: 0.9836 - val_loss: 0.8335 - val_accuracy: 0.8262
Epoch 6/10
508/508 - 79s - loss: 0.0556 - accuracy: 0.9814 - val_loss: 0.8342 - val_accuracy: 0.8255
Epoch 7/10
508/508 - 78s - loss: 0.0543 - accuracy: 0.9833 - val_loss: 0.8400 - val_accuracy: 0.8250
Epoch 8/10
508/508 - 79s - loss: 0.0514 - accuracy: 0.9824 - val_loss: 0.8474 - val_accuracy: 0.8252
Epoch 9/10
508/508 - 78s - loss: 0.0494 - accuracy: 0.9847 - val_loss: 0.8598 - val_accuracy: 0.8227
Epoch 10/10
508/508 - 79s - loss: 0.0455 - accuracy: 0.9853 - val_loss: 0.8624 - val_accura